In [ ]:
from echopype.convert import Convert
from echopype import Process
import glob
import random
import matplotlib.pyplot as plt

import numpy as np


from glider_azfp import load_glider, merge_glider_AZFP

In [ ]:
gl = load_glider(dataset_id ='ru32-20190102T1317-profile-sci-rt')

# gl

# load an AZFP file

you'd want to build the loop below here

In [ ]:
path_to_raw = './data/'

# alternativly, pick a random file
all_files = glob.glob(path_to_raw+'*.01A')
# a random file from the list (got to remove path and extention)
fname = random.choice(all_files)[7:-4]

# path to the 'fake' calibration file
calib_file = './data/NLB_test.XML'

print('random file used:')
print(fname + '.01A')

In [ ]:

# fname = '19011711'
# fname = '19011717'
fname = '19011709' #decent swarm, maybe bottom reflection?


# converts the raw AZFP data
# we are 'tricking' echopype with a fake .XML file
# the real files fail because there is no temperature data
# edit the './data/' parts so the path works for where you put the files
data_tmp = Convert(path_to_raw + fname + '.01A', calib_file)


# place to save processed files
savedir = './data_netcdfs/'

# converts to a netcdf we can use with xarray
data_tmp.raw2nc(save_path = savedir )


# read the converted data 

nc_path = savedir + fname + '.nc'

# process so you can get calibrated, clean, data 

ed = Process(nc_path)   # create a processing object

# need to figure out how much it matter that these vary a lot

ed.temperature = gl.potential_temperature.mean().values  
ed.salinity = gl.salinity.mean().values     
ed.pressure = gl.pressure.mean().values     # pressure in dbar
ed.recalculate_environment()  # recalculate related parameters

ed.calibrate()           # Sv
ed.remove_noise(SNR=2)        # denoise, this is adaptive! will change with the file! NOT WORKING NOW!

azfp = ed.Sv_clean


In [ ]:
ds = merge_glider_AZFP( gl, azfp, )

# ds

In [ ]:

plt.figure(figsize=(10,10))
ds.Sv[0,:,:].plot(x="ping_time", y="bin_depths", yincrease=False)
ds.bottom_depth.plot(marker='o', ls='')


# processing with echopy

try some:
1. seabed masking
1. impluse noise removal
1. swarm detection

In [ ]:
from echopy.processing import mask_seabed as  maskSB
from echopy.processing import mask_impulse as mIN
from echopy.processing import mask_shoals as mSH


In [ ]:
# sb = mask_seabed.deltaSv(ds.Sv[0,:,:].data.T, ds.range[0,:,0].data,
#                       r0=10, thr=-20)

# sb = mask_seabed.maxSv(ds.Sv[0,:,:].data.T, ds.range[0,:,0].data)

sb = maskSB.ariza(ds.Sv[0,:,:].data.T, ds.range[0,:,0].data,
                      dc=3)

# sb = mask_seabed.experimental(ds.Sv[0,:,:].data.T, ds.range[0,:,0].data,
#                       )

plt.pcolormesh( ds.ping_time, ds.range[0,:,0], sb)

plt.figure()
plt.pcolormesh(ds.ping_time, ds.range[0,:,0],ds.Sv[0,:,:].T.data )

plt.figure()

ds.Sv[0,:,:].where(sb.T==0).plot(x="ping_time")

In [ ]:
# look for swarms:

# shoal = mSH.weill(ds.Sv[0,:,:].data.T, thr=-70, maxvgap=5, minvlen=1, minhlen=5 )

# sv_no_imp = mIN.wang(ds.Sv[0,:,:].data.T, thr=(-70,-40), erode=[(3,3)],
#                       dilate=[(7,7)], median=[(7,7)])[0]


# plt.pcolormesh(ds.ping_time, ds.range[0,:,0],sv_no_imp )

shoal = mSH.echoview(ds.Sv[0,:,:].data.T,
    np.arange( len(ds.Sv.range_bin )+1),
    np.arange(len(ds.Sv.ping_time)+1),
    thr=-85,
    mincan=(5, 5),
    maxlink=(30, 2),
    minsho=(30, 2),
)


plt.figure()
plt.pcolormesh(ds.ping_time, ds.range[0,:,0], shoal[0] )